In [1]:
%pip install transformers datasets torch scikit-learn accelerate

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('LightTai/personalized-email')

In [3]:
print(dataset)
print(dataset['train'][:5])
df = dataset['train'].to_pandas()
print(df.head())
print(df.shape)

DatasetDict({
    train: Dataset({
        features: ['product', 'gender', 'profession', 'hobby', 'email'],
        num_rows: 30
    })
})
{'product': ['piano lessons', 'guitar lessons', 'vacation plans', 'vacation plans', 'vacation plans'], 'gender': ['male', 'male', 'male', 'female', 'female'], 'profession': ['college students', 'college students', 'college students', 'college students', 'company employees'], 'hobby': ['like to play piano', 'like to play piano', 'like swimming', 'like to look at the scenery', 'like to look at the scenery'], 'email': ["Subject: Elevate Your Piano Skills - Exclusive Offer Inside!\n\nHey [Name],\n\nLooking to unlock your piano potential? As a fellow male college student and a passionate piano player, I understand your love for music. That's why I'm thrilled to offer you exclusive piano lessons designed to fit your busy student schedule.\n\nMaster your favorite melodies, refine techniques, and gain a deeper understanding of music theory-all while enjoyin

In [14]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset

model_checkpoint = "postbot/distilgpt2-emailgen"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    features = [f"{prod} {gen} {prof} {hob}" 
                for prod, gen, prof, hob in zip(examples["product"], 
                                                examples["gender"], 
                                                examples["profession"], 
                                                examples["hobby"])]
    tokenized_inputs = tokenizer(features, truncation=True, padding="max_length", max_length=512)

    # Tokenize the email column which is our target
    tokenized_targets = tokenizer(examples["email"], truncation=True, padding="max_length", max_length=512)

    tokenized_inputs['labels'] = tokenized_targets['input_ids']  # Assign target token ids as labels for training
    return tokenized_inputs


# Assuming 'df' is your DataFrame and it's already loaded
train_df, test_df = train_test_split(df, test_size=0.2)

# Convert the DataFrames back to Hugging Face dataset format if necessary
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["product", "gender", "profession", "hobby", "email"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["product", "gender", "profession", "hobby", "email"])
# Assuming 'dataset' is a Hugging Face 'datasets' object
# tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["product", "gender", "profession", "hobby", "email"])


Python(10990) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10991) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10992) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10993) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Map (num_proc=4):   0%|          | 0/24 [00:00<?, ? examples/s]

Python(10994) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10995) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10996) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10997) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(10998) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Map (num_proc=4):   0%|          | 0/6 [00:00<?, ? examples/s]

Python(10999) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [15]:
print(tokenized_train_dataset)

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 24
})


In [16]:
tokenized_train_dataset[1]

{'__index_level_0__': 13,
 'input_ids': [25119,
  12437,
  4257,
  4152,
  2444,
  588,
  284,
  711,
  1830,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,

In [7]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

training_args = TrainingArguments(
    output_dir="./model_output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
)

trainer.train()

# Save the model and tokenizer after training is complete
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 4.001028060913086, 'eval_runtime': 35.4448, 'eval_samples_per_second': 0.169, 'eval_steps_per_second': 0.056, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 4.108323097229004, 'eval_runtime': 34.9715, 'eval_samples_per_second': 0.172, 'eval_steps_per_second': 0.057, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 3.9420006275177, 'eval_runtime': 29.9241, 'eval_samples_per_second': 0.201, 'eval_steps_per_second': 0.067, 'epoch': 3.0}
{'train_runtime': 1060.4061, 'train_samples_per_second': 0.068, 'train_steps_per_second': 0.017, 'train_loss': 5.850269317626953, 'epoch': 3.0}


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/merges.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [17]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
print(eval_results)

  0%|          | 0/2 [00:00<?, ?it/s]

Perplexity: 32.74
{'eval_loss': 3.4884700775146484, 'eval_runtime': 55.1655, 'eval_samples_per_second': 0.109, 'eval_steps_per_second': 0.036, 'epoch': 3.0}


In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

# Set the random seed for reproducible results
set_seed(42)

# Load the model and tokenizer, ensuring pad_token and eos_token are correctly set
model_checkpoint = "./saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Set the tokenizer's padding direction to the left
tokenizer.padding_side = 'left'

def generate_email(product, gender, profession, hobby):
    # Prepare the input data
    input_text = f"{product} {gender} {profession} {hobby}"
    
    # Tokenize the input data
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    
    # Generate the output, adding parameters to control the generation process
    outputs = model.generate(**inputs, 
                             max_new_tokens=100,  # Set to generate 100 new tokens
                             temperature=0.7,     # Moderately control randomness
                             top_k=50,            # Choose only from the top 50 tokens by probability
                             top_p=0.95,          # Cumulative probability threshold is 0.95
                             no_repeat_ngram_size=2)  # Avoid repetition of 2-token n-grams
    
    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

# Example usage
email_text = generate_email("Laptop", "Male", "Software Engineer", "Cycling")
print(email_text)

# Laptop Male Software Engineer Cycling Team
# <NUMBER>-<PHONE>
# Email: <EMAIL> | Twitter: @Cycling_Team
# Cyclists are invited to participate in the Cycling Cycling World Cup in Brazil.
# The Cycling Federation of Brazil is a global organization dedicated to promoting cycling 
# and the development of sustainable living. Cycling is an international organization that promotes the health, 
# safety and well-being of all people. The Cycling Foundation is dedicated solely to the pursuit of the highest quality 
# and quality of life

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Laptop Male Software Engineer Cycling Team
<NUMBER>-<PHONE>
Email: <EMAIL> | Twitter: @Cycling_Team
Cyclists are invited to participate in the Cycling Cycling World Cup in Brazil.
The Cycling Federation of Brazil is a global organization dedicated to promoting cycling and the development of sustainable living. Cycling is an international organization that promotes the health, safety and well-being of all people. The Cycling Foundation is dedicated solely to the pursuit of the highest quality and quality of life
